In [1]:
import torch
from transformers import AutoTokenizer, BertForMaskedLM, LongformerForMaskedLM
import numpy as np

/Users/tglim/miniforge3/envs/aiinfra/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1. BERT

In [2]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForMaskedLM.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
inputs = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)

for tokens in inputs.input_ids:
    token_len = 0
    for token in tokens:
        if token != tokenizer.pad_token_id:
            token_len += 1
            continue
        break
    tokens[np.random.randint(1, token_len - 2)] = tokenizer.mask_token_id

with torch.inference_mode():
    logits = model(**inputs).logits

sentence_index, mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)

predicted_token_id = []
for index, sentence in enumerate(sentence_index):
    if sentence >= len(predicted_token_id):
        predicted_token_id.append([])

    predicted_token_id[-1].append(tokenizer.decode(logits[sentence, mask_token_index[index]].argmax(axis=-1)))

In [4]:
tokenizer.batch_decode(inputs.input_ids)

['[CLS] The capital of [MASK] is Paris. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] The capital of France is [MASK]. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] The capital of France is [MASK]. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 우리나라의 수도는 Seoul [MASK]. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 강남에는 외 [MASK]인이 많이 보인다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] Java [MASK]ript 는 웹개발에 필수적인 programming 언어이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] 나 오늘 미 [MASK]실에 다녀와서 기분이 좋

In [5]:
print(logits.shape)
print(sentence_index, mask_token_index)
print(predicted_token_id)

torch.Size([9, 30, 119547])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]) tensor([ 4,  6,  6,  8,  5,  2,  5,  7, 19])
[['France'], ['Paris'], ['Paris'], ['##이다'], ['##국'], ['##sc'], ['##사'], ['푸'], ['##임']]


In [6]:
labels = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)['input_ids']

labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)

In [7]:
print(labels.shape)
print(outputs.loss)

torch.Size([9, 30])
tensor(2.6566, grad_fn=<NllLossBackward0>)


# 2. KoBERT

In [8]:
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertForMaskedLM.from_pretrained('skt/kobert-base-v1')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
inputs = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)

for tokens in inputs.input_ids:
    token_len = 0
    for token in tokens:
        if token != tokenizer.pad_token_id:
            token_len += 1
            continue
        break
    tokens[np.random.randint(1, token_len - 2)] = tokenizer.mask_token_id

with torch.inference_mode():
    logits = model(**inputs).logits

sentence_index, mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)

predicted_token_id = []
for index, sentence in enumerate(sentence_index):
    if sentence >= len(predicted_token_id):
        predicted_token_id.append([])

    predicted_token_id[-1].append(tokenizer.decode(logits[sentence, mask_token_index[index]].argmax(axis=-1)))

In [10]:
tokenizer.batch_decode(inputs.input_ids)

['[CLS] The capital of Fr[MASK]ce is Paris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] [MASK] capital of France is Paris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] The capit[MASK] of France is Paris.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] 우리나라의[MASK]는 Seoul 이다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS][MASK]에는 외국인이 많이 보인다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] Javascript 는 웹개발에 필수적인 pro[MASK]ramming 언어이다.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] 나 오늘 미용실에 다녀와서 기분이 좋아. 다음[MASK] 너도 같이 가자.[SEP][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD][PAD]',
 '[CLS] [MASK]끔 오타가 날 수 이[UNK]지만 너른 마음으로 양해바랍니

In [11]:
print(logits.shape)
print(sentence_index, mask_token_index)
print(predicted_token_id)

torch.Size([9, 37, 8002])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]) tensor([11,  2,  7,  3,  1, 19, 15,  2, 17])
[['쑤'], ['컵'], ['뜯'], ['보이'], ['잣'], ['크로스'], ['텁'], ['캡'], ['빠']]


In [12]:
labels = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)['input_ids']

labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)

In [13]:
print(labels.shape)
print(outputs.loss)

torch.Size([9, 37])
tensor(10.9358, grad_fn=<NllLossBackward0>)


# 3. LongFormer
* requirement: pip protobuf 버전 3.20.0 ~ 3.20.3

In [2]:
MAX_SEQUENCE_LENGTH = 4096

tokenizer = AutoTokenizer.from_pretrained(
    'AshtonIsNotHere/xlm-roberta-long-base-4096',
    max_length=MAX_SEQUENCE_LENGTH,
    padding="max_length",
    truncation=True,
)
model = LongformerForMaskedLM.from_pretrained(
    'AshtonIsNotHere/xlm-roberta-long-base-4096',
    max_length=MAX_SEQUENCE_LENGTH
)

Some weights of the model checkpoint at AshtonIsNotHere/xlm-roberta-long-base-4096 were not used when initializing LongformerForMaskedLM: ['longformer.embeddings.position_ids']
- This IS expected if you are initializing LongformerForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
inputs = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)

for tokens in inputs.input_ids:
    token_len = 0
    for token in tokens:
        if token != tokenizer.pad_token_id:
            token_len += 1
            continue
        break
    tokens[np.random.randint(1, token_len - 2)] = tokenizer.mask_token_id

with torch.inference_mode():
    logits = model(**inputs).logits

sentence_index, mask_token_index = (inputs.input_ids == tokenizer.mask_token_id).nonzero(as_tuple=True)

predicted_token_id = []
for index, sentence in enumerate(sentence_index):
    if sentence >= len(predicted_token_id):
        predicted_token_id.append([])

    predicted_token_id[-1].append(tokenizer.decode(logits[sentence, mask_token_index[index]].argmax(axis=-1)))

In [4]:
tokenizer.batch_decode(inputs.input_ids)

['<s> The capital<mask> France is Paris.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s><mask> capital of France is Paris.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> The capital of<mask> is Paris.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> 우리나라의 수도는<mask> 이다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> 강남<mask> 외국인이 많이 보인다.</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> Javascript 는 웹개발에 필수적인<mask>ng 언어이다.</s><pad><pad><pad><pad><pad><pad><pad><pad>',
 '<s> 나 오늘 미용실에 다녀와서 기분이 좋아. 다음<mask> 너도 같이 가자.</s>',
 '<s> 가끔 오타가 날 수 이ᄊ지만 너른 마음<mask> 양해바랍니다.</s><pad>',
 '<s> 당신 혹시 React.js 와 Node.js<mask>라는 프레임워크에 대해 잘 알아요?</s>']

In [5]:
print(logits.shape)
print(sentence_index, mask_token_index)
print(predicted_token_id)

torch.Size([9, 23, 250002])
tensor([0, 1, 2, 3, 4, 5, 6, 7, 8]) tensor([ 3,  1,  4,  5,  2,  9, 15, 15, 12])
[['of'], ['The'], ['France'], ['서울'], ['에서는'], ['programmi'], ['에'], ['으로'], ['']]


In [6]:
labels = tokenizer([
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    'The capital of France is Paris.',
    '우리나라의 수도는 Seoul 이다.',
    '강남에는 외국인이 많이 보인다.',
    'Javascript 는 웹개발에 필수적인 programming 언어이다.',
    '나 오늘 미용실에 다녀와서 기분이 좋아. 다음엔 너도 같이 가자.',
    '가끔 ㅇㅗ타가 날 수 이ㅆ지만 너른 마음으로 양해바랍니다.',
    '당신 혹시 React.js 와 Node.js 라는 프레임워크에 대해 잘 알아요?',
], return_tensors='pt', padding=True)['input_ids']

labels = torch.where(inputs.input_ids == tokenizer.mask_token_id, labels, -100)

outputs = model(**inputs, labels=labels)

In [7]:
print(labels.shape)
print(outputs.loss)

torch.Size([9, 23])
tensor(1.3794, grad_fn=<NllLossBackward0>)


# test for adding tokens to dictionary

In [ ]:
# tokenizer.add_tokens('React')
# newEmbedding = model.get_input_embeddings()
# newEmbedding.weight.dtype
# newEmbedding.weight = torch.nn.Parameter(torch.concat([newEmbedding.weight, torch.zeros((1, 768))], dim=0))
# model.set_input_embeddings(newEmbedding)
# newEmbedding